# Module 07: Subagents and Orchestration

**Difficulty**: ⭐⭐⭐ Advanced  
**Estimated Time**: 180 minutes  
**Prerequisites**: [Module 03 - Skills](03_working_with_skills.ipynb), [Module 04 - Commands](04_custom_slash_commands.ipynb), [Module 05 - Hooks](05_hooks_automation.ipynb)

## Learning Objectives

By the end of this module, you will be able to:

1. Understand subagent architecture and stateless nature
2. Use the Task tool to delegate work to specialized agents
3. Create custom subagent definitions with specific tool permissions
4. Run multiple agents in parallel for faster workflows
5. Design multi-agent orchestration patterns
6. Understand agent tool permissions and security

---

## 1. Introduction to Subagents

### What are Subagents?

**Subagents** are specialized instances of Claude that handle specific tasks. Think of them as **expert assistants** you can delegate work to.

**Main conversation**: You (the user) talking to Claude  
**Subagent**: Claude spawns a specialized assistant for a specific task

```
You → Claude (Main)
        ↓ Delegates
        Code Reviewer Agent → Reviews code
        ↓ Returns result
You ← Claude (Main) ← Review report
```

### Why Delegate to Specialized Agents?

**Benefits**:

1. **Focused expertise**: Agents specialized for specific tasks
2. **Parallel execution**: Multiple agents work simultaneously
3. **Cleaner separation**: Each agent has its own context
4. **Tool restrictions**: Limit what each agent can do
5. **Scalability**: Break complex tasks into manageable pieces

**Example**: PR review workflow
```
Main Claude:
  ├─ Agent 1: Review code quality → Report A
  ├─ Agent 2: Check security → Report B
  └─ Agent 3: Test coverage → Report C
  
Combine reports → Final review
```

### Subagent vs Main Conversation

| Feature | Main Conversation | Subagent |
|---------|------------------|----------|
| **Context** | Full conversation history | Only task-specific context |
| **Tools** | All available tools | Restricted by definition |
| **Lifespan** | Entire session | Single task only |
| **Communication** | Back-and-forth with user | One-way: task → result |
| **Purpose** | General assistance | Specific task execution |

### Stateless Nature of Subagents

**Key concept**: Each subagent is **stateless** and **independent**.

```
Subagent invocation 1:
  Input: "Review file A"
  Output: "Report for file A"
  [Agent terminates]

Subagent invocation 2:
  Input: "Review file B"
  Output: "Report for file B"
  [Agent has NO memory of file A]
```

**Implication**: You can't have back-and-forth conversations with subagents. They:
- Receive a task
- Execute it
- Return results
- Terminate

---

## 2. Subagent Architecture

### Agent Definitions

Subagents are defined in `.claude/agents/` directory:

```
.claude/
├── agents/
│   ├── code-reviewer.md        → Code review agent
│   ├── test-generator.md       → Test generation agent
│   ├── doc-writer.md           → Documentation agent
│   └── security-auditor.md     → Security audit agent
└── settings.json
```

### Agent Definition File Format

**Structure** (`.claude/agents/agent-name.md`):

```markdown
# Agent Name

## Description
Brief description of what this agent does.

## Tools
List of tools this agent can use:
- Read
- Grep
- Glob

## Instructions
Detailed instructions for the agent:

1. What to do
2. How to do it
3. What format to return

## Output Format
Specify the exact output format expected.
```

### Tool Access Specification

**Available tools** you can grant to agents:

**File operations**:
- `Read`: Read files
- `Write`: Create new files
- `Edit`: Modify existing files
- `Glob`: Find files by pattern
- `Grep`: Search file contents

**Code execution**:
- `Bash`: Run shell commands

**Special**:
- `WebFetch`: Fetch web content
- `Task`: Delegate to other agents (nested)

**Security principle**: Give agents **only the tools they need**.

**Example**:
```markdown
## Tools
- Read     # Can read files
- Grep     # Can search code
- Glob     # Can find files
# NO Write, Edit, Bash - read-only agent!
```

### Agent Instructions and Goals

**Good instructions**:
- Clear, step-by-step process
- Specific criteria for evaluation
- Expected output format
- Error handling guidance

**Example**:
```markdown
## Instructions

You are a code quality reviewer. For each file:

1. **Read the file** using the Read tool
2. **Analyze for**:
   - Variable naming clarity
   - Function length (<50 lines)
   - Code duplication
   - Comment quality
3. **Rate each aspect** 1-5 stars
4. **Provide specific suggestions** for improvement

## Output Format

Return a structured report:
- File: [filename]
- Overall score: [X/5]
- Issues found: [count]
- Top 3 improvements:
  1. [Specific suggestion with line number]
  2. [Specific suggestion with line number]
  3. [Specific suggestion with line number]
```

### Communication Patterns

**One-way communication**:
```
Main Claude → Task tool → Subagent
                           ↓
                        Execute
                           ↓
Main Claude ← Result ← Subagent
```

**No back-and-forth**: Subagent can't ask questions. Provide all context upfront!

---

## 3. Using the Task Tool

### Built-in Task Tool Overview

The **Task tool** is how you delegate work to subagents.

**Syntax** (in Claude Code):
```
User: "Review the authentication module"

Claude (main): I'll use the Task tool to delegate this to a code reviewer agent.
               
               [Uses Task tool]
               - subagent_type: "code-reviewer"
               - prompt: "Review auth.py for code quality issues"
               
               [Agent executes and returns report]
               
               Here's the review from the code reviewer agent:
               [Shows report to user]
```

### Delegating Work to Agents

**When Claude uses Task tool**:
1. Identifies a suitable subagent (by type)
2. Crafts a specific task prompt
3. Invokes the subagent
4. Waits for results
5. Processes and presents results to you

**You don't directly call Task tool** - Claude decides when to use it based on:
- Agent availability
- Task complexity
- Performance benefits

### Specifying subagent_type

The `subagent_type` parameter tells Claude which agent to use:

```python
# In agent definition: .claude/agents/code-reviewer.md
# Claude sees the filename and can use subagent_type="code-reviewer"
```

**Built-in agent types** (in Claude Code):
- `general-purpose`: General tasks
- `Explore`: Fast codebase exploration
- `Plan`: Planning and design tasks

**Custom agent types**: Defined by your `.md` files in `.claude/agents/`

### Receiving Results

**Subagent returns**: A single message with results.

**Main Claude receives**: The complete agent output.

**User sees**: Main Claude's interpretation/summary of results.

```
Flow:
  Subagent → [Detailed report] → Main Claude
                                     ↓
                              [Processes]
                                     ↓
  User ← [Formatted summary] ← Main Claude
```

---

## 4. Hands-On Section 1: Using Built-in Agents ⭐

Let's explore how to use built-in agents effectively.

### Exercise 1: Explore Agent for Codebase Navigation

**Scenario**: You want to understand how error handling works in your project.

**Without agents** (manual search):
```python
# You would manually:
# 1. Search for "try" and "except"
# 2. Read each file
# 3. Analyze patterns
# 4. Summarize findings
```

**With Explore agent**:
```
User: "How is error handling implemented across the codebase?"

Claude: [Uses Task tool with Explore agent]
        The Explore agent found:
        - 15 files with try/except blocks
        - Main patterns: custom exceptions in errors.py
        - Logging integrated in utils/logger.py
        - [Detailed analysis...]
```

**Benefits**:
- Faster than manual search
- Comprehensive analysis
- Structured output

### Exercise 2: Delegating Testing Tasks

**Example prompt**:
```
User: "Find all Python files that don't have corresponding test files"

Claude: [Delegates to Explore agent]
        
        Found 8 Python modules without tests:
        1. src/utils/parser.py
        2. src/models/user.py
        3. src/api/endpoints.py
        ...
```

### Exercise 3: Observe Agent Behavior

Let's create a simple agent tracking script:

In [ ]:
from pathlib import Path
import json

def list_available_agents():
    """List all custom agents defined in .claude/agents/"""
    
    agents_dir = Path('.claude/agents')
    
    if not agents_dir.exists():
        print("❌ No .claude/agents directory found")
        print("   Create it to define custom agents")
        return
    
    agent_files = list(agents_dir.glob('*.md'))
    
    if not agent_files:
        print("ℹ️  No custom agents defined yet")
        print("   Create .md files in .claude/agents/")
        return
    
    print(f"🤖 Available Custom Agents ({len(agent_files)}):\n")
    
    for agent_file in sorted(agent_files):
        agent_name = agent_file.stem
        content = agent_file.read_text()
        
        # Extract description (first paragraph)
        lines = content.split('\n')
        description = "No description"
        for line in lines:
            if line.strip() and not line.startswith('#'):
                description = line.strip()[:80]
                break
        
        # Extract tools
        tools = []
        in_tools = False
        for line in lines:
            if '## Tools' in line:
                in_tools = True
                continue
            if in_tools:
                if line.startswith('##'):
                    break
                if line.strip().startswith('-'):
                    tool = line.strip().lstrip('- ').split('#')[0].strip()
                    if tool:
                        tools.append(tool)
        
        print(f"  📋 {agent_name}")
        print(f"     Description: {description}")
        if tools:
            print(f"     Tools: {', '.join(tools)}")
        print()

# List available agents
list_available_agents()

**Key Observations**:
- Agents are autonomous during execution
- They use tools according to their permissions
- Results come back in one message

---

## 5. Creating Custom Subagents

### Agent Definition Template

**Basic template** for `.claude/agents/your-agent.md`:

```markdown
# Agent Name

Brief one-line description of the agent's purpose.

## Description

More detailed description:
- What this agent does
- When to use it
- What it's good at

## Tools

List of tools this agent can access:
- Read
- Grep
- Glob

## Instructions

Step-by-step instructions for the agent:

1. **First**, do this
2. **Then**, analyze for:
   - Criteria 1
   - Criteria 2
3. **Finally**, compile results

## Output Format

Specify exactly what format to return:

```
# Results for [Task]

## Summary
[Brief summary]

## Findings
1. [Finding 1]
2. [Finding 2]

## Recommendations
- [Recommendation 1]
- [Recommendation 2]
```
```

### Description and Purpose

**Good descriptions**:
- Clear purpose statement
- Specific use cases
- What makes this agent special

**Example**:
```markdown
## Description

This agent performs comprehensive code reviews focusing on:
- Code quality and maintainability
- Adherence to project conventions
- Potential bugs and edge cases

Use this agent when:
- Reviewing pull requests
- Auditing existing code
- Establishing quality baselines

The agent is read-only and won't modify files.
```

### Tool Access Definition

**Principle**: Grant minimum necessary tools.

**Read-only agent** (code reviewer):
```markdown
## Tools
- Read    # Read file contents
- Grep    # Search for patterns
- Glob    # Find files
```

**Write agent** (test generator):
```markdown
## Tools
- Read    # Read source files
- Write   # Create test files
- Grep    # Search for functions
- Glob    # Find source files
```

**Powerful agent** (refactoring):
```markdown
## Tools
- Read    # Read files
- Edit    # Modify files
- Grep    # Search code
- Glob    # Find files
- Bash    # Run tests after changes
```

### Output Format Guidance

**Why specify output format?**
- Consistent, parseable results
- Easy to combine multiple agent outputs
- Helps agent focus on structure

**Good output format**:
```markdown
## Output Format

Return results in this exact format:

```json
{
  "file": "path/to/file.py",
  "score": 8.5,
  "issues": [
    {
      "line": 42,
      "severity": "medium",
      "message": "Function too long (65 lines)",
      "suggestion": "Split into smaller functions"
    }
  ],
  "summary": "Overall code quality is good..."
}
```
```

---

## 6. Hands-On Section 2: Code Reviewer Agent ⭐⭐

### Exercise 4: Create Code Reviewer Agent

Let's create a comprehensive code review agent.

**File**: `.claude/agents/code-reviewer.md`

In [ ]:
from pathlib import Path

# Create agents directory
agents_dir = Path('.claude/agents')
agents_dir.mkdir(parents=True, exist_ok=True)

# Code reviewer agent definition
code_reviewer_content = '''# Code Quality Reviewer

Reviews code for quality, maintainability, and best practices.

## Description

This agent performs comprehensive code reviews focusing on:
- **Code Quality**: Readability, naming, structure
- **Best Practices**: Language-specific conventions
- **Potential Issues**: Bugs, edge cases, performance
- **Maintainability**: Documentation, complexity

Use this agent when:
- Reviewing pull requests
- Auditing code quality
- Establishing baselines
- Training developers

The agent is read-only and won't modify files.

## Tools

- Read
- Grep
- Glob

## Instructions

For each file provided:

1. **Read the file** completely using the Read tool

2. **Analyze for quality**:
   - Variable and function naming (descriptive? consistent?)
   - Function length (ideally <50 lines)
   - Code duplication (DRY principle)
   - Comments (helpful? not obvious?)
   - Error handling (proper try/except?)
   - Magic numbers (should be constants?)

3. **Check best practices**:
   - Language-specific conventions (PEP 8 for Python, etc.)
   - Proper imports organization
   - Type hints (for Python 3.5+)
   - Docstrings for public functions

4. **Identify issues**:
   - Potential bugs (off-by-one, null checks, etc.)
   - Security vulnerabilities (SQL injection, XSS, etc.)
   - Performance concerns (N+1 queries, inefficient loops)
   - Edge cases not handled

5. **Rate the code** on a scale of 1-10:
   - 1-3: Needs significant improvement
   - 4-6: Acceptable but could be better
   - 7-8: Good quality
   - 9-10: Excellent, exemplary code

6. **Provide actionable feedback**:
   - Specific line numbers for issues
   - Concrete suggestions for improvement
   - Code examples when helpful

## Output Format

Return a structured review in this format:

```markdown
# Code Review: [filename]

## Overall Score: X/10

## Summary
[2-3 sentence summary of code quality]

## Strengths
- [What the code does well]
- [Good practices observed]

## Issues Found

### High Priority
1. **Line X**: [Issue description]
   - Problem: [What's wrong]
   - Suggestion: [How to fix]

### Medium Priority
1. **Line Y**: [Issue description]

### Low Priority (Code style)
1. **Line Z**: [Style issue]

## Recommendations
1. [Top recommendation]
2. [Second recommendation]
3. [Third recommendation]
```
'''

# Write the agent definition
reviewer_file = agents_dir / 'code-reviewer.md'
reviewer_file.write_text(code_reviewer_content)

print("✅ Created code-reviewer agent")
print(f"   Location: {reviewer_file}")
print("\n📋 Agent capabilities:")
print("   - Read files")
print("   - Search code (Grep)")
print("   - Find files (Glob)")
print("   - Read-only (no modifications)")
print("\n🎯 Usage:")
print('   Ask Claude: "Review the authentication module"')
print('   Claude will use the code-reviewer agent automatically')

### Exercise 5: Test the Code Reviewer

Create a sample file to review:

In [ ]:
# Create a sample Python file with some issues
sample_code = '''def calc(x, y):
    # TODO: add error handling
    return x / y

def process_data(data):
    result = []
    for i in range(len(data)):
        if data[i] > 10:
            result.append(data[i] * 2)
    return result

# Global variable (not recommended)
API_KEY = "secret_key_12345"

def fetch_user(id):
    query = "SELECT * FROM users WHERE id = " + str(id)  # SQL injection risk!
    # execute query...
    return None
'''

sample_file = Path('sample_code.py')
sample_file.write_text(sample_code)

print("✅ Created sample_code.py with intentional issues:")
print("   - No error handling (division by zero)")
print("   - Non-descriptive function names")
print("   - Inefficient loop (should use list comprehension)")
print("   - Hardcoded secret")
print("   - SQL injection vulnerability")
print("\n🧪 Test: Ask Claude to review this file using the code-reviewer agent")

**Expected behavior**: Claude will delegate to the code-reviewer agent, which will:
1. Read sample_code.py
2. Identify all the issues
3. Provide a structured review
4. Give specific recommendations

---

## 7. Hands-On Section 3: Test Generator Agent ⭐⭐⭐

### Exercise 6: Create Test Generator Agent

Let's create an agent that generates unit tests.

**File**: `.claude/agents/test-generator.md`

In [ ]:
test_generator_content = '''# Unit Test Generator

Generates comprehensive unit tests for Python code.

## Description

This agent analyzes source code and generates unit tests with:
- **Happy path** test cases
- **Edge cases** (empty input, None, extreme values)
- **Error cases** (invalid input, exceptions)
- **Boundary conditions** (min/max values)

Uses appropriate testing framework:
- Python: pytest or unittest
- JavaScript: Jest
- Java: JUnit

## Tools

- Read
- Write
- Grep
- Glob

## Instructions

When asked to generate tests for a file:

1. **Read the source file** completely

2. **Identify testable units**:
   - Public functions and methods
   - Classes and their methods
   - Complex logic branches

3. **For each function**, generate tests for:
   - **Normal operation**: Expected inputs → expected outputs
   - **Edge cases**: 
     - Empty strings, lists, dicts
     - None values
     - Zero, negative numbers
     - Very large values
   - **Error cases**:
     - Invalid types
     - Out of range values
     - Missing required parameters
   - **Boundary conditions**: Min/max acceptable values

4. **Generate test code** with:
   - Clear test names (test_function_name_condition_expected)
   - Arrange-Act-Assert pattern
   - Docstrings explaining what's being tested
   - Mock external dependencies
   - Proper setup and teardown if needed

5. **Follow best practices**:
   - Each test tests ONE thing
   - Tests are independent (can run in any order)
   - Use descriptive assertions
   - Include comments for complex scenarios

6. **Create test file** with standard naming:
   - source.py → test_source.py
   - module.py → test_module.py

## Output Format

Generate a complete test file with:

```python
"""Tests for [module name]."""

import pytest  # or unittest
from module import function_to_test


class TestFunctionName:
    """Tests for function_name()."""
    
    def test_normal_case(self):
        """Test function with typical inputs."""
        # Arrange
        input_data = ...
        
        # Act
        result = function_to_test(input_data)
        
        # Assert
        assert result == expected
    
    def test_edge_case_empty_input(self):
        """Test function with empty input."""
        ...
    
    def test_error_case_invalid_type(self):
        """Test function raises error for invalid type."""
        with pytest.raises(TypeError):
            function_to_test("invalid")
```

Include a summary comment at the top listing:
- Number of test cases generated
- Coverage summary (what's tested)
- Any functions that couldn't be tested (and why)
'''

test_gen_file = agents_dir / 'test-generator.md'
test_gen_file.write_text(test_generator_content)

print("✅ Created test-generator agent")
print(f"   Location: {test_gen_file}")
print("\n📋 Agent capabilities:")
print("   - Read source files")
print("   - Write test files")
print("   - Search for functions (Grep)")
print("   - Find source files (Glob)")
print("\n🎯 Usage:")
print('   Ask Claude: "Generate tests for sample_code.py"')
print('   Agent will create test_sample_code.py with comprehensive tests')

### Exercise 7: Test the Test Generator

Now ask Claude to generate tests for the sample code:

```
User: "Generate unit tests for sample_code.py"

Claude: [Delegates to test-generator agent]
        
        I've generated comprehensive tests in test_sample_code.py:
        - 12 test cases total
        - Covers happy path, edge cases, and error conditions
        - Includes tests for SQL injection vulnerability
        - All tests follow pytest conventions
```

**Expected test file structure**:
```python
def test_calc_normal():
    assert calc(10, 2) == 5

def test_calc_division_by_zero():
    with pytest.raises(ZeroDivisionError):
        calc(10, 0)

def test_process_data_filters_correctly():
    result = process_data([5, 15, 8, 20])
    assert result == [30, 40]
```

---

## 8. Parallel Agent Execution

### Running Multiple Agents Simultaneously

**Key insight**: Agents can run **in parallel** for independent tasks.

**Sequential execution** (slow):
```
Agent 1: Review code     (takes 30s)
  ↓ wait
Agent 2: Generate tests  (takes 30s)
  ↓ wait
Agent 3: Check security  (takes 30s)

Total time: 90 seconds
```

**Parallel execution** (fast):
```
Agent 1: Review code     ┐
Agent 2: Generate tests  ├─ All run simultaneously
Agent 3: Check security  ┘

Total time: ~30 seconds (limited by slowest agent)
```

### Faster Completion for Independent Tasks

**When to use parallel execution**:
- Tasks are **independent** (don't depend on each other's results)
- Tasks take **significant time**
- You want **faster overall completion**

**When NOT to use parallel**:
- Tasks are **dependent** (B needs results from A)
- Tasks are **quick** (overhead not worth it)
- Results need **specific ordering**

### Aggregating Results

After parallel agents complete:

```python
# Pseudo-code for how Claude handles this
results = {
    'code_review': agent1_result,
    'test_coverage': agent2_result,
    'security_scan': agent3_result
}

# Combine into final report
final_report = combine_results(results)
```

### Use Cases for Parallel Execution

**PR Review**:
```
Parallel:
  ├─ Code quality review
  ├─ Security scan
  ├─ Test coverage analysis
  └─ Documentation check

Combine → Comprehensive PR review
```

**Codebase Analysis**:
```
Parallel:
  ├─ Find all API endpoints
  ├─ Analyze database queries
  ├─ Map dependencies
  └─ Find unused code

Combine → Architecture report
```

**Multi-file Processing**:
```
Parallel:
  ├─ Review file1.py
  ├─ Review file2.py
  ├─ Review file3.py
  └─ Review file4.py

Combine → Overall quality report
```

---

## 9. Hands-On Section 4: Parallel Workflow ⭐⭐⭐

### Exercise 8: Run 3 Agents in Parallel

Let's create a comprehensive code analysis workflow.

**Scenario**: Analyze a codebase with three independent checks:
1. Code quality review
2. Test coverage analysis
3. Documentation completeness

First, let's create the third agent (documentation checker):

In [ ]:
doc_checker_content = '''# Documentation Checker

Analyzes code documentation completeness and quality.

## Description

This agent checks for:
- Function and class docstrings
- Parameter documentation
- Return value documentation
- Module-level documentation
- README files
- Inline comments quality

## Tools

- Read
- Grep
- Glob

## Instructions

For each file:

1. **Read the file** and identify:
   - All functions and classes
   - Public vs private members
   - Complex logic blocks

2. **Check documentation presence**:
   - Does each public function have a docstring?
   - Do classes have docstrings?
   - Are parameters documented?
   - Are return values documented?
   - Are exceptions documented?

3. **Evaluate documentation quality**:
   - Is it clear and concise?
   - Does it explain WHY, not just WHAT?
   - Are examples provided?
   - Is it up-to-date with code?

4. **Calculate coverage**:
   - % of public functions documented
   - % of classes documented
   - Overall documentation score

## Output Format

```markdown
# Documentation Analysis: [filename]

## Coverage: X%

- Functions documented: X/Y (Z%)
- Classes documented: X/Y (Z%)
- Parameters documented: X/Y (Z%)

## Missing Documentation
1. Function `func_name` (line X): No docstring
2. Class `ClassName` (line Y): Missing parameter docs

## Quality Issues
1. Line X: Docstring too vague
2. Line Y: Missing return value documentation

## Recommendations
1. [Priority improvement]
2. [Second improvement]
```
'''

doc_checker_file = agents_dir / 'doc-checker.md'
doc_checker_file.write_text(doc_checker_content)

print("✅ Created doc-checker agent")
print(f"   Location: {doc_checker_file}")
print("\n🤖 We now have 3 agents:")
print("   1. code-reviewer: Quality analysis")
print("   2. test-generator: Create tests")
print("   3. doc-checker: Documentation analysis")

### Exercise 9: Request Parallel Analysis

**Prompt to Claude**:
```
"Please analyze sample_code.py using three agents in parallel:
1. Code quality review
2. Test coverage analysis  
3. Documentation completeness

Give me a combined report."
```

**What happens**:
```
Claude (main):
  ├─ Spawns code-reviewer agent     ┐
  ├─ Spawns test-generator agent    ├─ Run in parallel
  └─ Spawns doc-checker agent       ┘
       ↓ All complete
  Combines results
       ↓
  Returns unified report
```

### Exercise 10: Compare Execution Times

Let's simulate the time difference:

In [ ]:
import time

def simulate_agent_execution():
    """Simulate parallel vs sequential agent execution."""
    
    # Simulated task durations (seconds)
    tasks = {
        'Code Review': 25,
        'Test Generation': 30,
        'Doc Check': 20
    }
    
    print("📊 Agent Execution Time Comparison\n")
    print("=" * 50)
    
    # Sequential execution
    print("\n🐌 Sequential Execution:")
    sequential_total = 0
    for task, duration in tasks.items():
        print(f"   {task}: {duration}s")
        sequential_total += duration
    print(f"\n   Total: {sequential_total}s")
    
    # Parallel execution
    print("\n⚡ Parallel Execution:")
    for task, duration in tasks.items():
        print(f"   {task}: {duration}s")
    parallel_total = max(tasks.values())
    print(f"\n   Total: {parallel_total}s (limited by slowest)")
    
    # Speedup
    speedup = sequential_total / parallel_total
    time_saved = sequential_total - parallel_total
    
    print("\n" + "=" * 50)
    print(f"\n✨ Results:")
    print(f"   Speedup: {speedup:.1f}x faster")
    print(f"   Time saved: {time_saved}s ({time_saved/sequential_total*100:.0f}%)")

simulate_agent_execution()

**Key Insight**: Parallel execution can provide 2-3x speedup for independent tasks!

---

## 10. Agent Orchestration Patterns

### Pattern 1: Sequential (One After Another)

**When to use**: Tasks depend on previous results.

```
Step 1: Analyze codebase → [File list]
  ↓
Step 2: Generate tests for files → [Test files]
  ↓
Step 3: Review generated tests → [Review report]
```

**Example**:
```
User: "Find all Python files without tests, generate tests, then review them"

Claude:
  1. [Explore agent] → Finds 5 files without tests
  2. [Test-gen agent] → Generates tests for all 5
  3. [Code-review agent] → Reviews generated tests
```

### Pattern 2: Parallel (Simultaneous)

**When to use**: Independent tasks, want speed.

```
┌─ Agent A: Task 1 ─┐
├─ Agent B: Task 2 ─┤ → All run simultaneously
└─ Agent C: Task 3 ─┘
        ↓
   Combine results
```

**Example**:
```
User: "Comprehensive analysis of the codebase"

Claude (parallel):
  ├─ [Code review] → Quality report
  ├─ [Security scan] → Security report
  ├─ [Performance analysis] → Performance report
  └─ [Dependency audit] → Dependency report
        ↓
   Combined analysis
```

### Pattern 3: Hierarchical (Nested Delegation)

**When to use**: Complex workflows with sub-workflows.

```
Main Agent
  ↓
Coordinator Agent
  ├─ Worker Agent 1
  ├─ Worker Agent 2
  └─ Worker Agent 3
```

**Example**:
```
User: "Prepare the codebase for release"

Main Claude:
  ↓
[Release Prep Agent]:
  ├─ [Test Agent] → Run all tests
  │   ├─ [Unit Test Agent]
  │   └─ [Integration Test Agent]
  ├─ [Doc Agent] → Update docs
  └─ [Version Agent] → Bump version
```

### Pattern 4: Fan-out/Fan-in

**When to use**: Process multiple items, combine results.

```
Input: [File1, File2, File3, File4]
        ↓ Fan out
    ┌───┼───┼───┐
    A1  A2  A3  A4  (Process each file)
    └───┼───┼───┘
        ↓ Fan in
  Combined Report
```

**Example**:
```
User: "Review all Python files in src/"

Claude:
  1. Find files → [file1.py, file2.py, ..., file10.py]
  2. Fan out: Review each in parallel
  3. Fan in: Combine into summary report
```

### Choosing the Right Pattern

| Scenario | Pattern | Why |
|----------|---------|-----|
| Step B needs output from Step A | Sequential | Dependency |
| 3 independent analyses | Parallel | Speed |
| Multi-stage pipeline | Hierarchical | Organization |
| Process 100 files | Fan-out/Fan-in | Scalability |
| Quick exploration | Single agent | Simplicity |

---

## 11. Best Practices

### 1. Clear, Focused Agent Tasks

**Good** ✅:
```markdown
Task: "Review auth.py for security vulnerabilities"
- Specific file
- Clear focus (security)
- Defined scope
```

**Bad** ❌:
```markdown
Task: "Check the code and make it better"
- Vague scope
- Unclear goal
- Hard to evaluate success
```

### 2. Limit Tool Access Appropriately

**Principle of Least Privilege**:

```markdown
# Read-only reviewer
Tools:
- Read ✅
- Grep ✅
- Glob ✅
- Write ❌ (doesn't need to modify)
- Bash ❌ (doesn't need to execute)

# Test generator
Tools:
- Read ✅ (read source)
- Write ✅ (create tests)
- Grep ✅ (find functions)
- Bash ❌ (doesn't need to run tests)
```

### 3. Structured Output Formats

**Why**: Makes results easy to parse and combine.

**Good** ✅:
```markdown
## Output Format

Return JSON:
{
  "file": "path/to/file",
  "score": 8.5,
  "issues": [{"line": 10, "msg": "..."}]
}
```

**Bad** ❌:
```markdown
Just describe the results in your own words.
```

### 4. Reusable Agent Designs

**Generic agents** that work with any input:

```markdown
# Code Reviewer (reusable)
Works with:
- Any Python file
- Any JavaScript file
- Auto-detects language
```

**vs**

```markdown
# Auth Module Reviewer (too specific)
Only works with:
- auth.py specifically
```

### 5. Testing Agents Individually

Before using agents in workflows:

1. **Test with simple inputs**
2. **Verify output format**
3. **Check tool usage**
4. **Validate edge cases**

**Test script example**:

In [ ]:
def validate_agent_definition(agent_file):
    """Validate an agent definition file."""
    
    if not agent_file.exists():
        return {"valid": False, "error": "File not found"}
    
    content = agent_file.read_text()
    issues = []
    
    # Check required sections
    required_sections = ['## Description', '## Tools', '## Instructions', '## Output Format']
    for section in required_sections:
        if section not in content:
            issues.append(f"Missing section: {section}")
    
    # Check if description is clear
    if '## Description' in content:
        desc_start = content.index('## Description')
        desc_section = content[desc_start:desc_start+200]
        if len(desc_section.strip()) < 50:
            issues.append("Description too short")
    
    # Check if tools are listed
    if '## Tools' in content:
        tools_start = content.index('## Tools')
        tools_end = content.index('##', tools_start + 10) if '##' in content[tools_start + 10:] else len(content)
        tools_section = content[tools_start:tools_end]
        if '-' not in tools_section:
            issues.append("No tools listed")
    
    return {
        "valid": len(issues) == 0,
        "issues": issues,
        "file": agent_file.name
    }

# Validate all agents
print("🔍 Validating Agent Definitions\n")
print("=" * 50)

for agent_file in agents_dir.glob('*.md'):
    result = validate_agent_definition(agent_file)
    
    if result['valid']:
        print(f"\n✅ {result['file']}: Valid")
    else:
        print(f"\n❌ {result['file']}: Issues found")
        for issue in result['issues']:
            print(f"   - {issue}")

print("\n" + "=" * 50)

---

## 12. Common Agent Patterns

### Code Review Agents

**Variations**:
- General code review
- Security-focused review
- Performance review
- Style/formatting review

**Tools**: Read, Grep, Glob

### Test Generation Agents

**Variations**:
- Unit test generator
- Integration test generator
- E2E test generator
- Test data generator

**Tools**: Read, Write, Grep, Glob

### Documentation Agents

**Variations**:
- API documentation generator
- README generator
- Code comment generator
- Tutorial/guide generator

**Tools**: Read, Write, Grep, Glob

### Refactoring Agents

**Variations**:
- Extract function/method
- Rename refactoring
- Simplify complex code
- Update to modern syntax

**Tools**: Read, Edit, Grep, Bash (for testing)

### Analysis Agents

**Variations**:
- Dependency analyzer
- Dead code detector
- API endpoint mapper
- Database query analyzer

**Tools**: Read, Grep, Glob

---

## 13. Real-World Examples

### Example 1: PR Review Workflow

**Workflow**: Comprehensive pull request review

```
User: "Review PR #45"

Main Claude:
  1. Get changed files from git
  2. Launch parallel agents:
     ├─ [Code Review Agent] → Quality analysis
     ├─ [Security Agent] → Security scan
     ├─ [Test Coverage Agent] → Check test coverage
     └─ [Doc Agent] → Check documentation
  3. Combine results
  4. Generate PR comment with:
     - Overall assessment
     - Issues by priority
     - Approval/request changes
```

### Example 2: Test Coverage Improvement

**Workflow**: Achieve 80% test coverage

```
User: "Improve test coverage to 80%"

Main Claude:
  1. [Analysis Agent] → Find files with <80% coverage
     Result: 12 files need tests
  
  2. [Test Gen Agents] (parallel) → Generate tests for all 12
  
  3. Run tests, check coverage
  
  4. [Review Agent] → Review generated tests for quality
  
  5. Report: "Coverage improved from 65% to 82%"
```

### Example 3: Code Quality Analysis

**Workflow**: Assess overall codebase quality

```
User: "Analyze codebase quality and create improvement plan"

Main Claude:
  1. [Explore Agent] → Map codebase structure
  
  2. Fan-out to multiple analyzers (parallel):
     ├─ [Code Quality] → Score each file
     ├─ [Complexity] → Cyclomatic complexity
     ├─ [Dependencies] → Dependency graph
     └─ [Duplication] → Find duplicated code
  
  3. Aggregate results
  
  4. [Plan Agent] → Create improvement roadmap
     - High-priority issues
     - Quick wins
     - Long-term improvements
```

### Example 4: Documentation Generation Pipeline

**Workflow**: Generate complete project documentation

```
User: "Generate comprehensive documentation"

Main Claude:
  1. Sequential workflow:
     
     a) [Analysis Agent] → Understand codebase
        - API endpoints
        - Data models
        - Core features
     
     b) Parallel doc generation:
        ├─ [API Doc Agent] → Generate API docs
        ├─ [README Agent] → Create README
        ├─ [Tutorial Agent] → Write getting started
        └─ [Architecture Agent] → Document architecture
     
     c) [Review Agent] → Check docs quality
     
     d) Write all docs to /docs directory
```

---

## 14. Practice Exercises

### Exercise 11: Create 3 Custom Agents ⭐⭐⭐

**Goal**: Build a complete agent toolkit.

**Task**: Create three agents:

1. **Security Auditor**
   - Scans for security vulnerabilities
   - Checks for hardcoded secrets
   - Identifies SQL injection risks
   - Tools: Read, Grep, Glob

2. **Performance Analyzer**
   - Finds inefficient code
   - Detects N+1 queries
   - Identifies memory leaks
   - Tools: Read, Grep, Glob

3. **Dependency Auditor**
   - Maps dependency graph
   - Finds unused dependencies
   - Checks for circular dependencies
   - Tools: Read, Grep, Glob

**Deliverable**: Three working agent definitions in `.claude/agents/`

### Exercise 12: Build a Parallel Workflow ⭐⭐⭐

**Goal**: Create a comprehensive codebase analysis.

**Task**: Design a workflow that uses 4+ agents in parallel:
- Code quality review
- Security audit
- Documentation check
- Test coverage analysis

**Steps**:
1. Create/verify all agent definitions
2. Test each agent individually
3. Run all in parallel on a sample codebase
4. Combine results into unified report

**Bonus**: Measure and compare parallel vs sequential execution time.

### Exercise 13: Hierarchical Agent System ⭐⭐⭐

**Goal**: Build a multi-level agent orchestration.

**Task**: Create a release preparation workflow:

```
Release Coordinator Agent
  ├─ Quality Gate Agent
  │   ├─ Test Runner
  │   ├─ Linter
  │   └─ Security Scanner
  ├─ Documentation Agent
  │   ├─ README Updater
  │   └─ Changelog Generator
  └─ Version Manager
      ├─ Version Bumper
      └─ Tag Creator
```

**Deliverable**: Complete workflow that prepares code for release.

### Exercise 14: Custom Domain Agent ⭐⭐⭐

**Goal**: Create an agent for your specific domain.

**Task**: Design and implement an agent for your work:
- Data science: Dataset validator
- Web dev: Accessibility checker
- DevOps: Infrastructure auditor
- ML: Model evaluator

**Requirements**:
- Clear, specific purpose
- Well-defined tool permissions
- Structured output format
- Comprehensive instructions
- Tested on real examples

---

## 15. Summary

### What You've Learned

In this module, you mastered:

✅ **Subagent Fundamentals**
- What subagents are and why they're useful
- Stateless nature of agents
- Subagent vs main conversation differences

✅ **Agent Architecture**
- Agent definition file format
- Tool access specification
- Instructions and output formats
- Communication patterns

✅ **Using the Task Tool**
- How to delegate work to agents
- Specifying subagent types
- Receiving and using results

✅ **Creating Custom Agents**
- Code reviewer agent (read-only)
- Test generator agent (read-write)
- Documentation checker agent
- Agent validation

✅ **Parallel Execution**
- Running multiple agents simultaneously
- 2-3x speedup for independent tasks
- Aggregating parallel results

✅ **Orchestration Patterns**
- Sequential: Dependencies between tasks
- Parallel: Independent tasks for speed
- Hierarchical: Nested delegation
- Fan-out/Fan-in: Process multiple items

✅ **Best Practices**
- Clear, focused agent tasks
- Minimal tool permissions
- Structured output formats
- Reusable agent designs
- Testing agents individually

### Key Takeaways

1. **Subagents enable specialization** - Expert assistants for specific tasks
2. **Parallel execution speeds up workflows** - 2-3x faster for independent tasks
3. **Tool restrictions enhance security** - Give only necessary permissions
4. **Structured outputs enable composition** - Easy to combine agent results
5. **Choose the right pattern** - Sequential, parallel, or hierarchical

### Agent Ecosystem

| Agent Type | Purpose | Tools | Output |
|------------|---------|-------|--------|
| Code Reviewer | Quality analysis | Read, Grep, Glob | Structured review |
| Test Generator | Create tests | Read, Write, Grep | Test files |
| Doc Checker | Documentation | Read, Grep, Glob | Coverage report |
| Security Auditor | Security scan | Read, Grep, Glob | Vulnerability list |
| Refactoring | Code improvements | Read, Edit, Bash | Improved code |

---

## 16. What's Next?

In **Module 08: Advanced Workflows**, you'll learn:

- 🔗 **Combining all components** - Skills + Commands + Hooks + MCP + Agents
- 🏗️ **Complex multi-step workflows** - Real-world automation
- 🔄 **Error handling and recovery** - Robust workflows
- ⚡ **Performance optimization** - Fast, efficient automation
- 📊 **Workflow monitoring** - Track and improve
- 🎯 **Production-ready patterns** - Battle-tested approaches

Module 08 brings everything together into complete, production-ready automation!

---

## 17. Additional Resources

### Official Documentation
- [Claude Code Task Tool Guide](https://docs.anthropic.com/claude-code/task-tool)
- [Subagent Best Practices](https://docs.anthropic.com/claude-code/subagents)
- [Agent Configuration](https://docs.anthropic.com/claude-code/configuration#agents)

### Community Resources
- [Agent Example Library](https://github.com/anthropics/claude-code-agents)
- [Agent Templates](https://github.com/topics/claude-code-agent)
- [Claude Code Discord](https://discord.gg/claude-developers)

### Agent Patterns
- Code review agent patterns
- Test generation patterns
- Documentation automation
- Multi-agent workflows

### Related Modules
- **Module 03**: [Working with Skills](03_working_with_skills.ipynb)
- **Module 04**: [Custom Slash Commands](04_custom_slash_commands.ipynb)
- **Module 05**: [Hooks and Automation](05_hooks_automation.ipynb)
- **Module 06**: [MCP Servers](06_mcp_servers_integrations.ipynb)
- **Module 08**: [Advanced Workflows](08_advanced_workflows.ipynb) (coming next)

---

## 🎉 Congratulations!

You've completed Module 07 and can now:
- Understand subagent architecture
- Create custom specialized agents
- Run agents in parallel for speed
- Design multi-agent workflows
- Apply orchestration patterns
- Follow agent best practices

**Next Step**: Move to Module 08 to learn advanced workflows that combine everything!

---

*Claude Code Mastery Series - Module 07 of 10*